In [70]:
import random
import torch
from torch import nn ,optim, Tensor
import torch.nn.functional as F
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn import GCNConv
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
import pandas as pd
from torch_geometric.utils import structured_negative_sampling

In [71]:
movie_path = '../data/ml-latest-small/movies.csv'
rating_path = '../data/ml-latest-small/ratings.csv'
print(pd.read_csv(movie_path).head())
print(pd.read_csv(rating_path).head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [72]:
def load_node_csv(path, index_col, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

In [73]:
from sentence_transformers import SentenceTransformer
class SequenceEncoder:
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()

In [74]:
class GenresEncoder:
    def __init__(self, sep='|'):
        self.sep = sep

    def __call__(self, df):
        genres = set(g for col in df.values for g in col.split(self.sep))
        mapping = {genre: i for i, genre in enumerate(genres)}

        x = torch.zeros(len(df), len(mapping))
        for i, col in enumerate(df.values):
            for genre in col.split(self.sep):
                x[i, mapping[genre]] = 1
        return x

In [75]:
movie_x, movie_mapping = load_node_csv(
    movie_path, index_col='movieId', encoders={
        'title': SequenceEncoder(),
        'genres': GenresEncoder()
    })

Batches: 100%|██████████| 305/305 [00:04<00:00, 65.32it/s]


In [76]:
user_x, user_mapping = load_node_csv(rating_path, index_col='userId')

In [77]:
from torch_geometric.data import HeteroData
data = HeteroData()
data['users'].num_nodes = len(user_mapping)  # Users do not have any features.
data['items'].x = movie_x

In [78]:
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    df = pd.read_csv(path, **kwargs)

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr

In [79]:
class IdentityEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [80]:
edge_index, edge_label = load_edge_csv(
    rating_path,
    src_index_col='userId',
    src_mapping=user_mapping,
    dst_index_col='movieId',
    dst_mapping=movie_mapping,
    encoders={'rating': IdentityEncoder(dtype=torch.long)},
)

data['user', 'rates', 'movie'].edge_index = edge_index
data['user', 'rates', 'movie'].edge_label = edge_label
print(data)

HeteroData(
  users={ num_nodes=610 },
  items={ x=[9742, 404] },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836, 1],
  }
)


In [81]:
edge_index, edge_label = load_edge_csv(
    rating_path,
    src_index_col='userId',
    src_mapping=user_mapping,
    dst_index_col='movieId',
    dst_mapping=movie_mapping,
    encoders={'rating': IdentityEncoder(dtype=torch.long)},
)

data['user', 'rates', 'movie'].edge_index = edge_index
data['user', 'rates', 'movie'].edge_label = edge_label

In [82]:
num_node = len(user_mapping)+len(movie_mapping)
user_emb = nn.Embedding(num_embeddings=len(user_mapping)+1, embedding_dim=movie_x.size(dim=1))
nn.init.normal_(user_emb.weight, std=0.1)
user_x=torch.arange(1,len(user_mapping)+1).view(1,-1)
user_x = torch.LongTensor(user_x)
user_x = user_emb(user_x)
user_x = user_x.squeeze(0)
data['users'].x = user_x

In [83]:
def sample_mini_batch(batch_size, edge_index):
    """
    Args:
        batch_size (int): 批大小
        edge_index (torch.Tensor): 2*N的边列表
    Returns:
        tuple: user indices, positive item indices, negative item indices
    """
   
    edges = structured_negative_sampling(edge_index)
    edges = torch.stack(edges, dim=0)
    indices = random.choices(
        [i for i in range(edges[0].shape[0])], k=batch_size)
    batch = edges[:, indices]
    user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
    return user_indices, pos_item_indices, neg_item_indices

In [84]:
# 定义GCN空域图卷积神经网络
class GCNConv(MessagePassing):
    # 网络初始化
    def __init__(self, k):
        """
        :param in_channels: 节点属性向量的维度
        :param out_channels: 经过图卷积之后，节点的特征表示维度
        """
        # 定义伽马函数为求和函数,aggr='add'
        super(GCNConv, self).__init__(aggr='add')
        # 定义最里面那个线性变换

 
    # 定义信息汇聚函数
    def message(self, x_j, norm):
        # 正则化
        # norm.view(-1,1)将norm变为一个列向量
        # x_j是节点的特征表示矩阵
        return norm.view(-1, 1) * x_j
 
    # 前向传递，进行图卷积
    def forward(self, data : HeteroData, edge_index):
        users_emb = data['users'].x
        items_emb = data['items'].x
        edge_index = edge_index
        
        return users_emb , items_emb

In [86]:
print(edge_index.shape)
edge_index_norm = gcn_norm(
            edge_index, add_self_loops=True)
print(edge_index_norm)

torch.Size([2, 100836])
(tensor([[   0,    0,    0,  ..., 9739, 9740, 9741],
        [   2,    5,   43,  ..., 9739, 9740, 9741]]), tensor([0.0094, 0.0068, 0.0048,  ..., 0.5000, 0.5000, 0.5000]))


In [ ]:
model = GCNConv(3)
xs= model.forward(data,edge_index)
print(xs)